In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pyro
import torch
import cellij
from pyro.nn import PyroModule
import pyro.distributions as dist
from cellij.core.models import MOFA


/Users/tim.treis/anaconda3/envs/spatialdata/envs/cellij/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Create data
import os
import cellij
import anndata
import muon as mu
import numpy as np
import pandas as pd

from importlib import resources
from collections import UserDict


mdata = cellij._data.Importer().load_CLL()
mdata["mrna"]

AnnData object with n_obs × n_vars = 136 × 5000

In [4]:
model = cellij.models.MOFA(n_factors=50)
model.add_data(data=mdata, na_stategy="knn_by_obs")

In [5]:
model.fit(likelihood="Normal",  epochs=1000, verbose_epochs=50)

ValueError: at site "data", invalid log_prob shape
  Expected [200, -1, -1], actual [0, 200, 1, 9627]
  Try one of the following fixes:
  - enclose the batched tensor in a with pyro.plate(...): context
  - .to_event(...) the distribution being sampled
  - .permute() data dimensions

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1, 1, figsize=(30, 10))
sns.heatmap(model._guide.locs.w.detach().numpy().squeeze(), center=0, cmap='RdBu_r')

In [ ]:
w = model._guide.locs.w.detach().numpy().squeeze()
z = model._guide.locs.z.detach().numpy().squeeze()

print(w.shape, z.shape)

xhat = np.matmul(z, w)

fig, ax = plt.subplots(1, 2, figsize=(30, 10))
sns.heatmap(xhat, center=0, cmap='RdBu_r', ax=ax[0])
sns.heatmap(mdata["mrna"].X, center=0, cmap='RdBu_r', ax=ax[1])

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(30, 10))
sns.heatmap(mdata.X.nu, center=0, cmap='RdBu_r')